In [1]:
import tensorflow as tf

import numpy as np

import sys

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [8]:
l = {'grammar1':'absd'}

In [10]:
l.update({'grammar2':'dfda'})

In [11]:
l

{'grammar1': 'absd', 'grammar2': 'dfda'}

In [2]:
##restricting no of gpus
gpus = tf.config.experimental.list_physical_devices('GPU')
device_to_use = gpus[3]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.experimental.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [3]:
tf.__version__

'2.0.0'

In [4]:
sys.version

'3.7.5 (default, Oct 25 2019, 15:51:11) \n[GCC 7.3.0]'

In [5]:
num_uids = 50
input_uid = tf.keras.Input(shape=(1,), dtype=tf.int32)
input_shared_features = tf.keras.layers.Input(shape=(128,), dtype=tf.float32)

class CustomLayer(tf.keras.layers.Layer):
    def __init__(self,num_uids):
        super(CustomLayer, self).__init__(trainable=True,dtype=tf.int64)
        self.num_uids = num_uids
        
    def build(self,input_shape):
        self.params = tf.Variable(tf.random.normal((num_uids, 9)), trainable=True)
        self.built=True
        
    def call(self, input_uid,input_shared_features):
        param = tf.gather_nd(self.params, input_uid)
        combined = tf.concat([param, input_shared_features], axis=-1)
        return combined
    
    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({'num_uids': self.num_uids})
        return config

combined = CustomLayer(num_uids)(input_uid,input_shared_features)
net = tf.keras.layers.Dense(128)(combined)
model = tf.keras.Model(inputs={'input_uid':input_uid,'input_shared_features':input_shared_features},outputs=net)

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
custom_layer (CustomLayer)      (None, 137)          450         input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          17664       custom_layer[0][0]               
Total params: 18,114
Trainable params: 18,114
Non-trainable params: 0
________________________

In [29]:
model.compile(optimizer=tf.optimizers.Adam(),
            loss=loss_fn,
            metrics=[metric_fn()])

ValueError: The model cannot be compiled because it has no loss to optimize.

In [28]:
model.predict([[10],np.ones(shape= (1,128))])

ValueError: could not broadcast input array from shape (128) into shape (1)

In [7]:
model_dir='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model/custom'
model.save(model_dir)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model/custom/assets


In [23]:
loaded_model = tf.keras.models.load_model(model_dir)

In [24]:
loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
custom_layer_2 (CustomLayer)    (None, 137)          450         input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          17664       custom_layer_2[0][0]             
Total params: 18,114
Trainable params: 18,114
Non-trainable params: 0
________________________

In [27]:
np.ones((1,128))

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [16]:
def train_one_step():
    pass

@tf.function#(input_signature=(tf.TensorSpec(shape=[],dtype=tf.float32),))
def train(num_steps):
    print("Tracing with num_steps = {}".format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()

train(num_steps=10)
train(num_steps=20)


Tracing with num_steps = 10
Tracing with num_steps = 20


In [6]:
import tensorflow as tf

print('tensorflow version ',tf.__version__)

vocab_path='vocab_test.txt'

model_dir_lookup='model/lookup'
model_dir_lookup2 = model_dir_lookup+'_2'

vocab_path='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/vocab_test.txt'

model_dir_lookup='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model/lookup'
model_dir_lookup2 = model_dir_lookup+'_2'


class VocabLookup(tf.keras.layers.Layer):
    def __init__(self,vocab_path):
        super(VocabLookup, self).__init__(trainable=False,dtype=tf.int64)
        self.vocab_path = vocab_path
    def build(self,input_shape):
        table_init = tf.lookup.TextFileInitializer(self.vocab_path,tf.string,tf.lookup.TextFileIndex.WHOLE_LINE,
                              tf.int64,tf.lookup.TextFileIndex.LINE_NUMBER)
        self.table = tf.lookup.StaticHashTable(table_init,-1)
        self.built=True
        
    def call(self, input_text):
        splitted_text = tf.strings.split(input_text).to_tensor()
        word_ids = self.table.lookup(splitted_text)
        return word_ids
    
    def get_config(self):
        config = super(VocabLookup, self).get_config()
        config.update({'vocab_path': self.vocab_path})
        return config


class VocabLookup2(tf.keras.layers.Layer):
    def __init__(self):
        super(VocabLookup2, self).__init__(trainable=False,dtype=tf.int64)
    def build(self,input_shape):
        self.keys=['hi','testing','lookup','in','tf']
        ##keeping values to start from 1 instead of zero to be consistent with the file based approach
        values=range(1,len(self.keys)+1)
        table_init = tf.lookup.KeyValueTensorInitializer(keys=self.keys,values=values)
        self.table = tf.lookup.StaticHashTable(table_init,-1)
        self.built=True
        
    def call(self, input_text):
        splitted_text = tf.strings.split(input_text).to_tensor()
        word_ids = self.table.lookup(splitted_text)
        return word_ids
    
    def get_config(self):
        config = super(VocabLookup2, self).get_config()
        config.update({'keys': self.keys})
        return config


input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')

lookup_out = VocabLookup(vocab_path=vocab_path)(input_text)
lookup_out2 = VocabLookup2()(input_text)

model_lookup = tf.keras.Model(inputs={'input_text':input_text},outputs=lookup_out)
model_lookup2 = tf.keras.Model(inputs={'input_text':input_text},outputs=lookup_out2)

print('predict from model1 ', model_lookup.predict(['hi testing lookup in tf randomtext']))
print('predict from model2 ',model_lookup2.predict(['hi testing lookup in tf randomtext']))

model_lookup.save(model_dir_lookup)
model_lookup_loaded = tf.keras.models.load_model(model_dir_lookup)
print('loaded model config 1 - \n',model_lookup_loaded.get_config(),'\n')
print('predict from loaded model1 ',model_lookup_loaded.predict(['hi testing lookup in tf randomtext']))

model_lookup2.save(model_dir_lookup2)
model_lookup_loaded2 = tf.keras.models.load_model(model_dir_lookup2)
print('loaded model config 2 - \n',model_lookup_loaded2.get_config(),'\n')
print('predict from loaded model2 ', model_lookup_loaded2.predict(['hi testing lookup in tf randomtext']))

tensorflow version  2.2.0
predict from model1  [[ 0  1  2  3  4 -1]]
predict from model2  [[ 1  2  3  4  5 -1]]
INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model/lookup/assets
loaded model config 1 - 
 {'name': 'model_2', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None,), 'dtype': 'string', 'sparse': False, 'ragged': False, 'name': 'input_text'}, 'name': 'input_text', 'inbound_nodes': []}, {'class_name': 'VocabLookup', 'config': {'name': 'vocab_lookup_5', 'trainable': False, 'dtype': 'int64', 'vocab_path': '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/vocab_test.txt'}, 'name': 'vocab_lookup_5', 'inbound_nodes': [[['input_text', 0, 0, {}]]]}], 'input_layers': {'input_text': ['input_text', 0, 0]}, 'output_layers': [['vocab_lookup_5', 0, 0]]} 

predict from loaded model1  [[ 0  1  2  3  4 -1]]
INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/model/lookup_

In [2]:
model_lookup.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_text (InputLayer)      [(None,)]                 0         
_________________________________________________________________
vocab_lookup (VocabLookup)   (None, None)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
